In [1]:
import os
import h5py
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# 필요 변수 설정

In [2]:
RAW_DATA_DIR = '../../input/raw_data' # 카카오에서 다운로드 받은 데이터의 디렉터리
PROCESSED_DATA_DIR = '../../input/processed_v14' # 전처리된 데이터가 저장될 디렉터리
VOCAB_DIR = PROCESSED_DATA_DIR + '/vocab' # 전처리에 사용될 사전 파일이 저장될 디렉터리

# 학습에 사용될 파일 리스트
train_file_list = [
    "train.chunk.01",
    "train.chunk.02",
    "train.chunk.03",
    "train.chunk.04",
    "train.chunk.05",
    "train.chunk.06",
    "train.chunk.07",
    "train.chunk.08",
    "train.chunk.09"
]

# 개발에 사용될 파일 리스트. 공개 리더보드 점수를 내는데 사용된다.
dev_file_list = [
    "dev.chunk.01"    
]

# 테스트에 사용될 파일 리스트. 파이널 리더보드 점수를 내는데 사용된다.
test_file_list = [
    "test.chunk.01",
    "test.chunk.02", 
]

# 파일명과 실제 파일이 위치한 디렉토리를 결합한다.
train_path_list = [os.path.join(RAW_DATA_DIR, fn) for fn in train_file_list]
dev_path_list = [os.path.join(RAW_DATA_DIR, fn) for fn in dev_file_list]
test_path_list = [os.path.join(RAW_DATA_DIR, fn) for fn in test_file_list]

# PROCESSED_DATA_DIR과 VOCAB_DIR를 생성한다.
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)
os.makedirs(VOCAB_DIR, exist_ok=True)

# 대회 데이터를 pandas.DataFrame으로 만들기

In [3]:
# path_list의 파일에서 col 변수에 해당하는 컬럼 값들을 가져온다.
def get_column_data(path_list, div, col):
    col_data = []
    for path in path_list:
        h = h5py.File(path, 'r')
        col_data.append(h[div][col][:])
        h.close()
    return np.concatenate(col_data)

# path_list의 파일에서 학습에 필요한 컬럼들을 DataFrame 포맷으로 반환한다.
def get_dataframe(path_list, div):
    pids = get_column_data(path_list, div, col='pid')
    products = get_column_data(path_list, div, col='product')
    brands = get_column_data(path_list, div, col='brand')
    makers = get_column_data(path_list, div, col='maker')
    models = get_column_data(path_list, div, col='model')
    prices = get_column_data(path_list, div, col='price')
    updttms = get_column_data(path_list, div, col='updttm')
    bcates = get_column_data(path_list, div, col='bcateid')
    mcates = get_column_data(path_list, div, col='mcateid')
    scates = get_column_data(path_list, div, col='scateid')
    dcates = get_column_data(path_list, div, col='dcateid')
    
    df = pd.DataFrame({'pid': pids, 'product':products, 'brand':brands, 'maker':makers, 
                                      'model':models, 'price':prices, 'updttm':updttms, 
                                      'bcateid':bcates, 'mcateid':mcates, 'scateid':scates, 'dcateid':dcates} )
    
    # 바이트 열로 인코딩 상품제목과 상품ID를 유니코드 변환한다.
    df['pid'] = df['pid'].map(lambda x: x.decode('utf-8'))
    df['product'] = df['product'].map(lambda x: x.decode('utf-8'))
    df['brand'] = df['brand'].map(lambda x: x.decode('utf-8'))
    df['maker'] = df['maker'].map(lambda x: x.decode('utf-8'))
    df['model'] = df['model'].map(lambda x: x.decode('utf-8')) 
    df['updttm'] = df['updttm'].map(lambda x: x.decode('utf-8'))     
    
    return df

In [4]:
%%time
train_df = get_dataframe(train_path_list, 'train')
dev_df = get_dataframe(dev_path_list, 'dev')
test_df = get_dataframe(test_path_list, 'test')

CPU times: user 23.5 s, sys: 6.77 s, total: 30.3 s
Wall time: 32.3 s


In [5]:
train_df.head(5)

,pid,product,brand,maker,model,price,updttm,bcateid,mcateid,scateid,dcateid
0,O4486751463,직소퍼즐 - 1000조각 바다거북의 여행 (PL1275),퍼즐라이프,상품상세설명 참조,퍼즐라이프 직소퍼즐 바다거북의 여행,16520,20180227091029,1,1,2,-1
1,P3307178849,[모리케이스]아이폰6S/6S+ tree farm101 - 다이어리케이스[바보사랑][...,바보사랑,MORY|해당없음,아이폰6S/6S+ tree farm101 - 다이어리케이스|아이폰6S/6S+,20370,20180429085019,3,3,4,-1
2,R4424255515,크리비아 기모 3부 속바지 GLG4314P,크리비아,,크리비아 기모 3부 속바지 GLG4314P,-1,20180426102314,5,5,6,-1
3,F3334315393,[하프클럽/잭앤질]남성 솔리드 절개라인 포인트 포켓 팬츠 31133PT002_NA,잭앤질,㈜크리스패션,[잭앤질] 남성 솔리드 절개라인 포인트 포켓 팬츠 31133PT002_NA,16280,20180422084512,7,7,8,-1
4,N731678492,코드프리혈당시험지50매/코드프리시험지/최장유효기간,,기타,SD코드프리혈당시험지[50매],-1,20180424071623,10,9,11,-1


# 학습에 불필요한 컬럼 제거

In [6]:
# Peter Norvig Quote: “More data beats clever algorithms, but better data beats more data.”

In [7]:
import json
# 카테고리 이름과 ID의 매핑 정보를 불러온다.
cate_json = json.load(open(os.path.join(RAW_DATA_DIR, 'cate1.json')))

# (이름, ID) 순서를 (ID, 이름)으로 바꾼 후 dictionary로 만든다.
bid2nm = dict([(cid, name) for name, cid in cate_json['b'].items()])
mid2nm = dict([(cid, name) for name, cid in cate_json['m'].items()])
sid2nm = dict([(cid, name) for name, cid in cate_json['s'].items()])
did2nm = dict([(cid, name) for name, cid in cate_json['d'].items()])

# dictionary를 활용해 카테고리 ID에 해당하는 카테고리 이름 컬럼을 추가한다.
train_df['bcatenm'] = train_df['bcateid'].map(bid2nm)
train_df['mcatenm'] = train_df['mcateid'].map(mid2nm)
train_df['scatenm'] = train_df['scateid'].map(sid2nm)
train_df['dcatenm'] = train_df['dcateid'].map(did2nm)

In [8]:
train_df[['pid', 'product', 'brand', 'maker', 'model', 'price', 'bcatenm', 'mcatenm', 'scatenm', 'dcatenm']].head(15)

,pid,product,brand,maker,model,price,bcatenm,mcatenm,scatenm,dcatenm
0,O4486751463,직소퍼즐 - 1000조각 바다거북의 여행 (PL1275),퍼즐라이프,상품상세설명 참조,퍼즐라이프 직소퍼즐 바다거북의 여행,16520,악기/취미/만들기,보드게임/퍼즐,직소/퍼즐,
1,P3307178849,[모리케이스]아이폰6S/6S+ tree farm101 - 다이어리케이스[바보사랑][...,바보사랑,MORY|해당없음,아이폰6S/6S+ tree farm101 - 다이어리케이스|아이폰6S/6S+,20370,휴대폰/액세서리,휴대폰액세서리,아이폰액세서리,
2,R4424255515,크리비아 기모 3부 속바지 GLG4314P,크리비아,,크리비아 기모 3부 속바지 GLG4314P,-1,언더웨어,보정언더웨어,속바지/속치마,
3,F3334315393,[하프클럽/잭앤질]남성 솔리드 절개라인 포인트 포켓 팬츠 31133PT002_NA,잭앤질,㈜크리스패션,[잭앤질] 남성 솔리드 절개라인 포인트 포켓 팬츠 31133PT002_NA,16280,남성의류,바지,일자면바지,
4,N731678492,코드프리혈당시험지50매/코드프리시험지/최장유효기간,,기타,SD코드프리혈당시험지[50매],-1,건강관리/실버용품,건강측정용품,혈당지,
5,J4094617432,아트박스 POOM/낭만창고 idk385-시원한 맥주 캬하~,,,아트박스 POOM/낭만창고 idk385-시원한 맥주 캬하~,-1,홈/인테리어/가드닝,벽지/시트지/스티커,포인트스티커,
6,V4742097320,데버스 뉴 캠핑 BBQ 글러브 DVC E1209N 캠핑 등산,,기타,기타,-1,등산/캠핑/낚시,취사용품,기타취사용품,
7,Z4154445264,엘르스포츠 여성 비키니2PCS ETFLB06NVY,엘르스포츠,기타,ETFLB06NVY,-1,구기/헬스/수영/스키,수영복,스포츠 비키니,
8,L2121928457,[패션플러스][GEOX][GEOX] 제옥스 GH-405 블랙펄 클러치백,제옥스,제옥스,,30750,가방/지갑/잡화,여성가방,여성 클러치백,
9,J2944368118,[아트박스 POOM/꾸밈] iz099-우럭아왜우럭,꾸밈,꾸밈,인테리어액자-iz099-우럭아왜우럭,25200,홈/인테리어/가드닝,액자,액자세트,


## 데이터 분석

### brand 컬럼 분석

In [9]:
def get_vc_df(df, col):    
    vc_df = df[col].value_counts().reset_index()
    vc_df.columns = [col, 'count']
    vc_df['percentage'] = (vc_df['count'] / vc_df['count'].sum())*100    
    return vc_df

In [10]:
vc_df = get_vc_df(train_df, 'brand')
vc_df.head(10)

,brand,count,percentage
0,,3930113,48.312243
1,상품상세설명 참조,153156,1.882722
2,바보사랑,66645,0.819256
3,기타,64144,0.788512
4,상세설명참조,35795,0.440022
5,없음,33603,0.413076
6,아디다스,32292,0.396960
7,나이키,30785,0.378435
8,아트박스,28518,0.350567
9,알수없음,26768,0.329055


### maker 컬럼 분석

In [11]:
vc_df = get_vc_df(train_df, 'maker')
vc_df.head(10)

,maker,count,percentage
0,,2196846,27.005472
1,기타,2009828,24.706490
2,상품상세설명 참조,442299,5.437110
3,상세페이지 참조,63792,0.784185
4,상세설명참조,37899,0.465886
5,상품상세설명참조,36389,0.447324
6,아디다스,25472,0.313123
7,상세설명참조 / 상세설명참조,21873,0.268881
8,[불명],20836,0.256134
9,상품상세정보 참조,19786,0.243226


### model 컬럼 분석

In [12]:
vc_df = get_vc_df(train_df, 'model')
vc_df.head(10)

,model,count,percentage
0,,2063425,25.365349
1,기타,1606243,19.745285
2,상품상세설명 참조,295674,3.634673
3,없음,86475,1.063023
4,상세페이지 참조,35049,0.430852
5,근조화환,19642,0.241456
6,상품상세정보 참조,18727,0.230208
7,상세설명참조 / 상세설명참조,11387,0.139979
8,아트박스,10067,0.123752
9,상품 상세설명 참조,9723,0.119523


### price

In [13]:
vc_df = get_vc_df(train_df, 'price')
vc_df.head(10)

,price,count,percentage
0,-1,5270821,64.793349
1,85500,8872,0.109062
2,10800,6522,0.080174
3,9000,6505,0.079965
4,13500,5885,0.072343
5,9900,5792,0.071200
6,94000,5083,0.062484
7,18000,4801,0.059018
8,7200,4698,0.057752
9,89300,4241,0.052134


# 최종 결정된 DataFrame

In [14]:
# 불필요한 컬럼을 제거한 DataFrame 생성
train_df = train_df[['pid', 'product', 'bcateid', 'mcateid', 'scateid', 'dcateid']]
dev_df = dev_df[['pid', 'product', 'bcateid', 'mcateid', 'scateid', 'dcateid']]
test_df = test_df[['pid', 'product', 'bcateid', 'mcateid', 'scateid', 'dcateid']]

In [15]:
train_df.head()

,pid,product,bcateid,mcateid,scateid,dcateid
0,O4486751463,직소퍼즐 - 1000조각 바다거북의 여행 (PL1275),1,1,2,-1
1,P3307178849,[모리케이스]아이폰6S/6S+ tree farm101 - 다이어리케이스[바보사랑][...,3,3,4,-1
2,R4424255515,크리비아 기모 3부 속바지 GLG4314P,5,5,6,-1
3,F3334315393,[하프클럽/잭앤질]남성 솔리드 절개라인 포인트 포켓 팬츠 31133PT002_NA,7,7,8,-1
4,N731678492,코드프리혈당시험지50매/코드프리시험지/최장유효기간,10,9,11,-1


# product 칼럼 전처리 하기

### 센텐스피스 모델 학습

In [16]:
import re

# 특수기호를 나열한 패턴 문자열을 컴파일하여 패턴 객체를 얻는다.
p = re.compile('[\!@#$%\^&\*\(\)\-\=\[\]\{\}\.,/\?~\+\'"|_:;><`┃]')

# 위의 패턴 문자열의 매칭되는 문자는 아래 코드를 통해서 빈공백으로 치환할 것이다.

# 문장의 특수기호 제거 함수
def remove_special_characters(sentence, lower=True):
    sentence = p.sub(' ', sentence) # 패턴 객체로 sentence 내의 특수기호를 공백문자로 치환한다.
    sentence = ' '.join(sentence.split()) # sentence 내의 두개 이상 연속된 빈공백들을 하나의 빈공백으로 만든다.
    if lower:
        sentence = sentence.lower()
    return sentence

# product 칼럼에 특수기호를 제거하는 함수를 적용한 결과를 반환한다.
train_df['product'] = train_df['product'].map(remove_special_characters)

train_df.head() # 특수기호가 제거된 train_df의 상단 5행만 출력

,pid,product,bcateid,mcateid,scateid,dcateid
0,O4486751463,직소퍼즐 1000조각 바다거북의 여행 pl1275,1,1,2,-1
1,P3307178849,모리케이스 아이폰6s 6s tree farm101 다이어리케이스 바보사랑 무료배송,3,3,4,-1
2,R4424255515,크리비아 기모 3부 속바지 glg4314p,5,5,6,-1
3,F3334315393,하프클럽 잭앤질 남성 솔리드 절개라인 포인트 포켓 팬츠 31133pt002 na,7,7,8,-1
4,N731678492,코드프리혈당시험지50매 코드프리시험지 최장유효기간,10,9,11,-1


In [17]:
import sentencepiece as spm # sentencepiece 모듈을 가져온다.

# product 칼럼의 상품명을 product.txt 파일명으로 저장한다.
with open(os.path.join(VOCAB_DIR, 'product.txt'), 'w') as f:
    f.write(train_df['product'].str.cat(sep='\n'))

# sentencepiece 모델을 학습시키는 함수이다.
def train_spm(txt_path, spm_path,
              vocab_size=32000, input_sentence_size=1000000):  
    # input_sentence_size: 개수 만큼만 학습데이터로 사용된다.
    # vocab_size: 사전 크기
    spm.SentencePieceTrainer.Train(
        f' --input={txt_path} --model_type=bpe'
        f' --model_prefix={spm_path} --vocab_size={vocab_size}'
        f' --input_sentence_size={input_sentence_size}'
        f' --shuffle_input_sentence=true'
    )

# product.txt 파일로 sentencepiece 모델을 학습 시킨다. 
# 학습이 완료되면 spm.model, spm.vocab 파일이 생성된다.
train_spm(txt_path=os.path.join(VOCAB_DIR, 'product.txt'), 
          spm_path=os.path.join(VOCAB_DIR, 'spm')) # spm 접두어

# 센텐스피스 모델 학습이 완료되면 product.txt는 삭제
os.remove(os.path.join(VOCAB_DIR, 'product.txt'))

# 필요한 파일이 제대로 생성됐는지 확인
for dirname, _, filenames in os.walk(VOCAB_DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../../input/processed_v14/vocab/spm.vocab
../../input/processed_v14/vocab/spm.model


### train_df 전처리

In [18]:
%%time
# 센텐스피스 모델을 로드한다.
sp = spm.SentencePieceProcessor()
sp.Load(os.path.join(VOCAB_DIR, 'spm.model'))

# product 칼럼의 상품명을 분절한 결과를 tokenized_product 칼럼에 저장한다.
train_df['tokens'] = train_df['product'].map(lambda x: " ".join(sp.EncodeAsPieces(x)) )

train_df[['product', 'tokens']].head()

CPU times: user 2min 8s, sys: 881 ms, total: 2min 8s
Wall time: 2min 8s


,product,tokens
0,직소퍼즐 1000조각 바다거북의 여행 pl1275,▁직소퍼즐 ▁1000 조각 ▁바다 거북 의 ▁여행 ▁pl 12 75
1,모리케이스 아이폰6s 6s tree farm101 다이어리케이스 바보사랑 무료배송,▁모리케이스 ▁아이폰 6 s ▁6 s ▁tree ▁farm 101 ▁다이어리케이스 ...
2,크리비아 기모 3부 속바지 glg4314p,▁크리비아 ▁기모 ▁3 부 ▁속바지 ▁gl g 43 14 p
3,하프클럽 잭앤질 남성 솔리드 절개라인 포인트 포켓 팬츠 31133pt002 na,▁하프클럽 ▁잭앤질 ▁남성 ▁솔리드 ▁절개라인 ▁포인트 ▁포켓 ▁팬츠 ▁311 33...
4,코드프리혈당시험지50매 코드프리시험지 최장유효기간,▁코드 프리 혈 당 시험 지 50 매 ▁코드 프리 시험 지 ▁최 장 유 효 기간


In [19]:
train_df['token_len'] = train_df['tokens'].map(lambda x: len(x[1:].split(chr(9601))))

In [20]:
#sent = '▁직소퍼즐 ▁1000 조각 ▁바다 거북 의 ▁여행 ▁pl 12 75'
#[i for i, word in enumerate(sent[1:].split('▁')) for _ in range(len(word.split()))]

In [21]:
train_df[train_df['token_len'] > 30]

,pid,product,bcateid,mcateid,scateid,dcateid,tokens,token_len
3448,Q4186376907,글로벌샵 tronsmart mega bluetooth 4 2 40w bluetoot...,25,36,236,-1,▁글로벌샵 ▁tr on sm art ▁mega ▁bluetooth ▁4 ▁2 ▁40...,31
6456,H4268182688,ponc tyler portable wireless bluetooth speaker...,25,36,236,-1,▁ponc ▁t yl er ▁portable ▁wireless ▁bluetooth ...,35
34174,H2336121793,mmc 레이디 테슬 케이스 아이폰 6 s 플러스 갤럭시 s 6 엣지 알파 노트 센스...,3,3,4,-1,▁mm c ▁레이디 ▁테슬 ▁케이스 ▁아이폰 ▁6 ▁s ▁플러스 ▁갤럭시 ▁s ▁6...,34
62388,M4218249965,디니인터 70mm 교체용 이어 pads cushion for 소니 mdr v150 ...,25,32,35,-1,▁디니인터 ▁70 mm ▁교체용 ▁이어 ▁pads ▁cushion ▁for ▁소니 ...,38
87970,N3885183788,글로벌샵 wireless bluetooth speaker zoee s1 outdoo...,25,36,154,-1,▁글로벌샵 ▁wireless ▁bluetooth ▁speaker ▁zo ee ▁s ...,32
...,...,...,...,...,...,...,...,...
8131750,N3911165162,주 쎄마통상 chill pal blue mesh cooling towel stay ...,13,257,2067,-1,▁주 ▁쎄마통상 ▁chill ▁pal ▁blue ▁mesh ▁cooling ▁tow...,36
8132110,R4726732444,카트남 cablex iphone 충전기 3pack 6ft nylon braided ...,3,164,728,-1,▁카트남 ▁cable x ▁iphone ▁충전기 ▁3 pack ▁6 ft ▁nylo...,32
8133284,X4580014952,도쿄 all 쇼퍼 숄더 marathon only article 10 off peri...,13,347,2016,-1,▁도쿄 ▁all ▁쇼퍼 ▁숄더 ▁mar ath on ▁only ▁art icle ▁...,41
8134321,Y4400166651,직구 샵 일본 galaxy note 8 필름 anacend 수 붙여 양질tpu 케이...,17,455,1845,-1,▁직구 ▁샵 ▁일본 ▁galaxy ▁note ▁8 ▁필름 ▁an ac end ▁수 ...,41


In [22]:
#pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 500)
#pd.set_option('min_rows', 200)

###  dev_df, test_df 전처리

In [23]:
# 특수기호를 공백문자로 치환
dev_df['product'] = dev_df['product'].map(remove_special_characters) 
# product 칼럼을 분절한 뒤 token_id로 치환
dev_df['tokens'] = dev_df['product'].map(lambda x: " ".join([str(token_id) for token_id in sp.EncodeAsPieces(x)]))

# 특수기호를 공백문자로 치환
test_df['product'] = test_df['product'].map(remove_special_characters) 
# product 칼럼을 분절한 뒤 token_id로 치환
test_df['tokens'] = test_df['product'].map(lambda x: " ".join([str(token_id) for token_id in sp.EncodeAsPieces(x)]))

# 전처리된 대회 데이터를 파일로 저장한다.

### 전처리가 완료된 train_df, dev_df, test_df를 파일로 저장한다.

In [24]:
# product, tokenized_product 칼럼을 제외한 칼럼만을 남긴다.
columns = ['pid', 'tokens',  'bcateid', 'mcateid', 'scateid', 'dcateid']
train_df = train_df[columns] 
dev_df = dev_df[columns] 
test_df = test_df[columns] 

# csv 포맷으로 저장한다.
train_df.to_csv(os.path.join(PROCESSED_DATA_DIR, 'train.csv'), index=False) 
dev_df.to_csv(os.path.join(PROCESSED_DATA_DIR, 'dev.csv'), index=False) 
test_df.to_csv(os.path.join(PROCESSED_DATA_DIR, 'test.csv'), index=False) 

### 용량이 큰 img_feat 컬럼은 별도의 hdf5 포맷의 파일로 저장

In [25]:
# image_feature는 데이터의 크기가 크므로 처리함수를 별도로 분리하였다.
def save_column_data(input_path_list, div, col, n_img_rows, output_path):
    # img_feat를 저장할 h5 파일을 생성
    h_out = h5py.File(output_path, 'w')    
    # 대회데이터의 상품개수 x 2048(img_feat 크기)로 dataset을 할당한다.
    h_out.create_dataset(col, (n_img_rows, 2048), dtype=np.float32)
    
    offset_out = 0
    
    # h5포맷의 대회데이터에서 img_feat 칼럼만 읽어서 h5포맷으로 다시 저장한다.
    for in_path in tqdm(input_path_list, desc=f'{div},{col}'):
        h_in = h5py.File(in_path, 'r')
        sz = h_in[div][col].shape[0]
        h_out[col][offset_out:offset_out+sz] = h_in[div][col][:]
        offset_out += sz
        h_in.close()
    h_out.close()


save_column_data(train_path_list, div='train', col='img_feat', n_img_rows=len(train_df), 
                 output_path=os.path.join(PROCESSED_DATA_DIR, 'train_img_feat.h5'))
save_column_data(dev_path_list, div='dev', col='img_feat', n_img_rows=len(dev_df), 
                 output_path=os.path.join(PROCESSED_DATA_DIR, 'dev_img_feat.h5'))
save_column_data(test_path_list, div='test', col='img_feat', n_img_rows=len(test_df), 
                 output_path=os.path.join(PROCESSED_DATA_DIR, 'test_img_feat.h5'))

# 파일이 제대로 생성됐는지 확인
for dirname, _, filenames in os.walk(PROCESSED_DATA_DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))


../../input/processed_v14/dev_img_feat.h5
../../input/processed_v14/train_img_feat.h5
../../input/processed_v14/test_img_feat.h5
../../input/processed_v14/dev.csv
../../input/processed_v14/train.csv
../../input/processed_v14/test.csv
../../input/processed_v14/vocab/spm.vocab
../../input/processed_v14/vocab/spm.model
